In [1]:
# DON'T CHANGE this part: import libraries
import numpy as np
import scipy
import json
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import re
import itertools

In [2]:
# DON'T CHANGE this part: read data path
train_set_path, valid_set_path, random_number = input().split()

 train.json valid.json 58


In [3]:
# TODO:
# 1. preprocess: converting text to lowercase, coverting number, tokenization, removing stopword, stemming
# 2. embedding: hitogram matrix
# 3. classifier using linear regression
# 4. accuracy (for metric)

#### Ví dụ cho phần báo cáo, nên báo cáo cho từng phần code để rõ ràng

Báo cáo phần tiền xử lý: dùng xyz để tách từ, ...

... Đối với những từ out-of-vocab (xuất hiện trong tập train nhưng không có ở tập valid), xử lý bằng cách ... 

    Load the dataset with provided path from the input.

In [4]:
def load_dataset(train_set_path,valid_set_path):
    with open(train_set_path) as f:
        train = json.load(f)
    with open(valid_set_path) as f:
        valid = json.load(f)
    return train, valid

    Converting numbers to 'num'.
    Using regular expression 'r'\d+' to list out all the number then convert them to interger to sort (big to small) and back to string to ready for replacing step.
    The reverse sorting step is important due to python built in string replace function, if not the number with small value will be replaced first, left a mess in a long digit number.

In [5]:
def numb2str(text):
    number_list = re.findall(r'\d+', text)
    number_list = [int(number) for number in number_list]
    number_list = sorted(number_list, reverse = True)
    number_list = [str(number) for number in number_list]
    for number in number_list:
        text = text.replace(number,'num')
    return text

    Preprocessing the trainning set.
    Use lower to low case the text and numb2str to convert them to 'num'.
    Use word_tokenize() to split the sentence to word array.
    Only keeps the word that is not in stop words.
    Stemming with PorterStemmer.
    Append each word to the list and make it a set.

In [6]:
def train_preprocess(dataset,text,stop_words):
    vocab = []
    for data in dataset:
        data[text] = data[text].lower()
        data[text] = numb2str(data[text])
        data[text] = word_tokenize(data[text])
        data[text] = [word for word in data[text] if word not in stop_words]
        data[text] = [ps.stem(word) for word in data[text]]
        [vocab.append(word) for word in data[text]]
        vocab_set = sorted(set(vocab))
    return dataset, vocab_set

    Preprocessing the testing set.
    Almost the same with train_preprocess but only keeps the word that already exists in the vocabulary set, the others are turn to 'unk'.

In [7]:
def test_preprocess(dataset,vocab,text,stop_words):
    for data in dataset:
        data[text] = data[text].lower()
        data[text] = numb2str(data[text])
        data[text] = word_tokenize(data[text])
        data[text] = [word for word in data[text] if word not in stop_words]
        data[text] = [ps.stem(word) if ps.stem(word) in vocab else 'unk' for word in data[text]]
    return dataset

    Embedding.
    Create a dictionary to store word appearence with the keys come from vocabulary and default value is set to 0.
    Due to using dictionary the counting step is much easier and it wont be affect from sequencetial. The counter automatically increases 1 at the key which is the same as the word. 'Unk' isn't exist in the vocabulary so it doesn't count.
    A sentence generates a histogram vector and histograms vectors generate a histogram matrix.

In [8]:
def document_embedding(dataset,text,vocab):
    hist_matrix = []
    for data in  dataset:
        word_count = dict.fromkeys(vocab,0)
        for word in data[text]:
            if word != 'unk':
                word_count[word] += 1
        hist_vec = list(word_count.values())
        hist_matrix.append(hist_vec)
    return np.array(hist_matrix)

    Label encoder.
    Basically the review score ('overall') is treated as a categorial feature, so dummy it is a good way to generate a vector with 5 spaces. The idea likes asking each vector spaces if they have or not, if yes put 1 to that space and ofcourse the others are no and getting 0 putting at. Sequencetially dumming all the score to generate the label matrix.
    

In [9]:
def label_encoder(dataset,target):
    y = []
    for data in dataset:
        review_score = [0]*5
        review_score[int(data[target]-1)] =  1
        y.append(review_score)
    return np.array(y)

    Fitting.
    Using OLS formular:
#### $$ w = A.b$$
#### $$A = (X^T.X)\dagger$$
#### $$b = (X^T.y) $$
    Taking in X as the histogram matrix and y as labels array. On this approach the weights will be the shape of ( vocabulary, number of label spaces). 

In [10]:
def linear_regression(X,y):
    A = np.linalg.pinv(X.T@X)
    b = X.T@y
    weights = A@b
    return weights

    Predict.
    Using theory formular: predict =weight_0+ weight_1.attribute_1 + weight_2.attribute_2 +weight_3.attribute_3 + ... + weight_n.attribute_n.
    The output is a vector with 5 spaces, pass it through the softmax function to know which has the highest probability then find that space and adding 1 due to index of an array starting from 0.

In [11]:
def predict(X,weights):
    predict = [weights[0]]
    for i in range(len(X)):
        res = X[i]*weights[i+1]
        predict.append(res)
    predict = np.sum(predict, axis= 0)
    predict_softmax = scipy.special.softmax(predict)
    final_predict = np.argmax(predict_softmax) +1
    return final_predict

    Accuracy.
    Compute accuracy score with the formular.

In [12]:
def accuracy_score(y_predict, y_true):
    total = len(y_predict)
    right = [1 if y_predict[i] == y_true[i] else 0 for i in range(total)]
    return sum(right)/total

    Initialize parameters.

In [13]:
text = 'reviewText'
review = 'overall'
stop_words = stopwords.words('english')
ps = PorterStemmer()

    Load the dataset then preprocessing.

In [14]:
train_set, valid_set = load_dataset(train_set_path, valid_set_path)
train_set, vocab =  train_preprocess(train_set,text,stop_words)
valid_set =  test_preprocess(valid_set,vocab,text,stop_words)

    Embedding document, label encoding and generating y_true.

In [15]:
X_train =  document_embedding(train_set,text,vocab)
X_valid =  document_embedding(valid_set,text,vocab)
y_train =  label_encoder(train_set,review)
y_valid = [int(data[review]) for data in valid_set]

    Adding Constant to the input matrix.

In [16]:
X_train = np.concatenate((np.ones(shape=(X_train.shape[0],1)),X_train),axis=1)

    Building model.

In [17]:
weights = linear_regression(X_train,y_train)

    Predicting.

In [18]:
y_predict = [predict(X,weights) for X in X_valid]

    Calculating accuray.

In [19]:
accuracy = accuracy_score(y_predict, y_valid)

    Printing stuff.
    The only chance to get Accuracy = 0.556 is when you dupplicate 2 times the bias (Tested).

In [20]:
print(valid_set[int(random_number)][text])
print(f'M2 - {accuracy}')

['great', 'project', ',', 'littl', 'hard', 'read', 'direct', ',', 'think', 'due', ',', 'direct', '.', 'happi', 'bought', 'book']
M2 - 0.484
